# The purpose of this notebook is to assemble a pipeline for preparing the data, transforming it and then training a model to predict house prices. 

We are using the now famous [Ames dataset](https://www.notion.so/Diccionario-de-Datos-y-hints-8f8613b67b4140f1940f67463c4a0ced#bc3273399294410083987b036aef2356). Our goal is to predict the SalePrice of a house given the rest of the parameters. 

### Imports. 

In [64]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score 
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error as mse 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer

In [51]:
pd.set_option('display.max_columns', 40)

# Data preprocessing. 

A couple of things had to be done when preparing the data. As explained in the link provided in the first cell, it is best to remove houses with a "Gr Liv Area" above 4,000 and houses sold in abnormal conditions. Other than that, the rest of this process consisted of iteratively finding which columns to keep and which to drop.

There are more than 70 columns, so we won't explain in full detail why we chose some and left others out. But the main idea was to look for a parsimonious, coherent and logical model to predict house prices. We were perhaps a bit ruthless on dropping most categorical variables for their potential to increase dimensionality. 

In [91]:
COLS_TO_DROP = ['MS Zoning', 
                'Lot Frontage',
                'Lot Shape',
                'Street',
                'Land Contour',
                'Utilities',
                'Lot Config',
                'Land Slope',
                'Neighborhood',
                'Alley', 
                'Mas Vnr Type', 
                'Mas Vnr Area',          
                'Bsmt Qual', 
                'Bsmt Cond', 
                'Bsmt Exposure', 
                'BsmtFin Type 1',
                'BsmtFin Type 2',
                'Electrical',
                #FirePlace Qu, 
                #'Garage Type', 
                'Garage Yr Blt', 
               # 'Garage Finish', 
               # 'Garage Qual',
               # 'Garage Cond',
                #'Pool QC', 
                'Fence', 
                'Misc Feature',
                'Condition 1', 
                'Condition 2',
                'Exterior 1st', 
                'Exterior 2nd', 
                'Heating', 
                #'Heating QC',
               'Roof Style',
               'Roof Matl',
               'Foundation',
               'Functional',
               'Fireplaces',
               'Paved Drive',  #maybe keep it
               'Year Remod/Add',
               '3Ssn Porch',
               'Pool Area', #we already have a categorical variable for pool
               'Mo Sold',
               'Misc Val',
               'Open Porch SF',
               'BsmtFin SF 2',
               'Wood Deck SF',
               'Enclosed Porch',
               'Screen Porch',
               'Sale Condition',
               'Sale Type'
               ]

In [92]:
#data = pd.read_csv('casas_entrena.csv')
#data.columns.values

In [93]:
#we want to clean the column names 
def clean_column(col):
    return col.lower().replace('/','_').replace(' ', '_')

In [94]:
def replacing_nans(data):
    """
    Function to fill the columns with Nan Values where it is convenient to keep them.
    Parameters:
    -----------
    data: pandas dataframe
    
    Returns:
    --------
    data: pandas dataframe.
    """
    data["pool_qc"] = data["pool_qc"].fillna("None") #Pool brings useful information
    data["pool_qc"] = np.where(data["pool_qc"] == "None", 0, 1)
    data["fireplace_qu"] = data["fireplace_qu"].fillna("None")
    #Garage information might be useful
    for col in ('garage_type', 'garage_finish', 'garage_qual', 'garage_cond'):
        data[col] = data[col].fillna('None')
    
    return data

In [95]:
def basic_preprocessing(path):
    """
    Una función básica para preprocesar los datos de entrenamiento. 
    Parameters:
    -------
    path: str
          path en tu compu donde está el dataset
    
    Returns:
    --------
    data: pandas dataframe
          Un dataframe listo para el pipeline de sklearn. 
    """
    data = pd.read_csv(path)
    data = data[data['Sale Condition'] == "Normal"]
    data = data.drop(columns = COLS_TO_DROP)
    data = data[data['Gr Liv Area'] < 4_000]
    data.rename(columns={col: clean_column(col) for col in data.columns.values}, 
                 inplace=True)
    data = replacing_nans(data)
    
    return data 

In [96]:
#this is what it looks like right now
data = basic_preprocessing('casas_entrena.csv')
data.head()

,ms_subclass,lot_area,bldg_type,house_style,overall_qual,overall_cond,year_built,exter_qual,exter_cond,bsmtfin_sf_1,bsmt_unf_sf,total_bsmt_sf,heating_qc,central_air,1st_flr_sf,2nd_flr_sf,low_qual_fin_sf,gr_liv_area,bsmt_full_bath,bsmt_half_bath,full_bath,half_bath,bedroom_abvgr,kitchen_abvgr,kitchen_qual,totrms_abvgrd,fireplace_qu,garage_type,garage_finish,garage_cars,garage_area,garage_qual,garage_cond,pool_qc,yr_sold,saleprice
0,120,3072,TwnhsE,1Story,7,5,2004,Gd,TA,1059,306,1365,Ex,Y,1548,0,0,1548,1,0,2,0,2,1,Gd,7,TA,Attchd,Fin,2,388,TA,TA,0,2006,225000
2,120,3013,TwnhsE,1Story,7,5,2005,Gd,TA,16,1346,1362,Ex,Y,1506,0,0,1506,0,0,2,0,2,1,Gd,6,Gd,Attchd,Fin,2,440,TA,TA,0,2006,213490
4,120,3196,TwnhsE,1Story,8,5,2003,Gd,TA,0,1273,1273,Ex,Y,1456,0,0,1456,0,0,2,0,2,1,Gd,7,TA,Attchd,Fin,2,400,TA,TA,0,2006,215000
6,20,3182,1Fam,1Story,7,5,2007,Gd,TA,0,1266,1266,Ex,Y,1266,0,0,1266,0,0,2,0,2,1,Gd,6,Gd,Attchd,Fin,2,388,TA,TA,0,2008,159895
7,120,3203,TwnhsE,1Story,7,5,2006,Gd,TA,16,1129,1145,Ex,Y,1145,0,0,1145,0,0,2,0,2,1,Gd,6,None,Attchd,Fin,2,437,TA,TA,0,2010,160000


In [97]:
data.shape

(1208, 36)

# Transforming the data. 

We have done a basic preprocessing of the data just for it to be a little bit cleaner and prepared for a ML model. Now we will focus on the transformation pipeline to be implemented later on. 

In [98]:
X = data.iloc[:,:-1].copy()
Y = data["saleprice"].copy()

In [99]:
#this dictionary will be useful for the "CombinedAttributesAdder" class
idx = {col: X.columns.get_loc(col) for col in X.columns}

In [100]:
idx["overall_qual"] #it tells you the index of any given column

4

### Label Encoder. 

Some categorical features —kitchen quality, for example— are arrenged in an ordering such that it may be useful to transform them with Sklearn's LabelEncoder() so that we can further manipulate them and combine them with other features. 

This has the added benefit of reducing the number of variables in the final model because we are not one-hot-encoding them into n or n-1 new variables. 

In [101]:
def encode_variables(data, cols):
    """
    Function to transform numerical features into a numerical ordering. 
    
    Parameters: 
    -----------
    data: pandas dataframe
    cols: list
          list of cols that you want to transform with the label encoder. 
    
    Returns:
    --------
    
    data: pandas dataframe
          A cleanear, happier dataframe :) 
    
    """
    for col in cols:
        le = LabelEncoder() 
        data[col] = le.fit_transform(list(data[col].values))
    return data

In [102]:
cols_to_encode = ['exter_qual', 'exter_cond', 'heating_qc', 'central_air',
                 'kitchen_qual', 'fireplace_qu', 'garage_qual']

In [103]:
X = encode_variables(X, cols_to_encode)

In [104]:
X.head()

,ms_subclass,lot_area,bldg_type,house_style,overall_qual,overall_cond,year_built,exter_qual,exter_cond,bsmtfin_sf_1,bsmt_unf_sf,total_bsmt_sf,heating_qc,central_air,1st_flr_sf,2nd_flr_sf,low_qual_fin_sf,gr_liv_area,bsmt_full_bath,bsmt_half_bath,full_bath,half_bath,bedroom_abvgr,kitchen_abvgr,kitchen_qual,totrms_abvgrd,fireplace_qu,garage_type,garage_finish,garage_cars,garage_area,garage_qual,garage_cond,pool_qc,yr_sold
0,120,3072,TwnhsE,1Story,7,5,2004,2,4,1059,306,1365,0,1,1548,0,0,1548,1,0,2,0,2,1,2,7,5,Attchd,Fin,2,388,4,TA,0,2006
2,120,3013,TwnhsE,1Story,7,5,2005,2,4,16,1346,1362,0,1,1506,0,0,1506,0,0,2,0,2,1,2,6,2,Attchd,Fin,2,440,4,TA,0,2006
4,120,3196,TwnhsE,1Story,8,5,2003,2,4,0,1273,1273,0,1,1456,0,0,1456,0,0,2,0,2,1,2,7,5,Attchd,Fin,2,400,4,TA,0,2006
6,20,3182,1Fam,1Story,7,5,2007,2,4,0,1266,1266,0,1,1266,0,0,1266,0,0,2,0,2,1,2,6,2,Attchd,Fin,2,388,4,TA,0,2008
7,120,3203,TwnhsE,1Story,7,5,2006,2,4,16,1129,1145,0,1,1145,0,0,1145,0,0,2,0,2,1,2,6,3,Attchd,Fin,2,437,4,TA,0,2010


## Next steps:

Arreglar lo de la alberca para que solo tenga un uno si sí tiene y un cero si no. Armar una función que me haga toda la transformación del preprocesamiento, desde separar las X y las Y's ya transformadas (X's con todo el pipeline y las Y's con lo de logaritmo). 

Queda pendiente agregar más interacciones, lo de los logaritmos e ir unificando el pipeline. Aunque no estaría de más entrenar otro modelo! 

# Interactions. 

This is the fun part. Now we get to add new variables and interactions between some of them. For example, it makes sense to add a variable that shows the interaction between total square feet and the overall quality of the house. 

In [90]:
X.columns.values

array(['ms_subclass', 'lot_area', 'bldg_type', 'house_style',
       'overall_qual', 'overall_cond', 'year_built', 'exter_qual',
       'exter_cond', 'bsmtfin_sf_1', 'bsmt_unf_sf', 'total_bsmt_sf',
       'heating_qc', 'central_air', '1st_flr_sf', '2nd_flr_sf',
       'low_qual_fin_sf', 'gr_liv_area', 'bsmt_full_bath',
       'bsmt_half_bath', 'full_bath', 'half_bath', 'bedroom_abvgr',
       'kitchen_abvgr', 'kitchen_qual', 'totrms_abvgrd', 'fireplace_qu',
       'garage_type', 'garage_finish', 'garage_cars', 'garage_area',
       'garage_qual', 'garage_cond', 'pool_qc', 'yr_sold', 'sale_type'],
      dtype=object)

In [105]:
X.head()

,ms_subclass,lot_area,bldg_type,house_style,overall_qual,overall_cond,year_built,exter_qual,exter_cond,bsmtfin_sf_1,bsmt_unf_sf,total_bsmt_sf,heating_qc,central_air,1st_flr_sf,2nd_flr_sf,low_qual_fin_sf,gr_liv_area,bsmt_full_bath,bsmt_half_bath,full_bath,half_bath,bedroom_abvgr,kitchen_abvgr,kitchen_qual,totrms_abvgrd,fireplace_qu,garage_type,garage_finish,garage_cars,garage_area,garage_qual,garage_cond,pool_qc,yr_sold
0,120,3072,TwnhsE,1Story,7,5,2004,2,4,1059,306,1365,0,1,1548,0,0,1548,1,0,2,0,2,1,2,7,5,Attchd,Fin,2,388,4,TA,0,2006
2,120,3013,TwnhsE,1Story,7,5,2005,2,4,16,1346,1362,0,1,1506,0,0,1506,0,0,2,0,2,1,2,6,2,Attchd,Fin,2,440,4,TA,0,2006
4,120,3196,TwnhsE,1Story,8,5,2003,2,4,0,1273,1273,0,1,1456,0,0,1456,0,0,2,0,2,1,2,7,5,Attchd,Fin,2,400,4,TA,0,2006
6,20,3182,1Fam,1Story,7,5,2007,2,4,0,1266,1266,0,1,1266,0,0,1266,0,0,2,0,2,1,2,6,2,Attchd,Fin,2,388,4,TA,0,2008
7,120,3203,TwnhsE,1Story,7,5,2006,2,4,16,1129,1145,0,1,1145,0,0,1145,0,0,2,0,2,1,2,6,3,Attchd,Fin,2,437,4,TA,0,2010


In [63]:
class CombinedAttributesAdder(BaseEstimator, TransformerMixin): 
    def __init__(self, house_condition = True): 
        self.house_condition = house_condition
    def fit(self, X, y=None):
        return self 
    def transform(self, X, y=None):
        qual_squared = X[:, idx["overall_qual"]] ** 2 #quality squared
        qual_m2 = X[:, idx["overall_qual"]] * X[:, idx["lot_area"]] #quality * sq. feet
        garage_int = X[:, idx["garage_area"]] * X[:, idx["garage_qual"]]
        total_sf = X[:, idx['total_bsmt_sf']] + X[:, idx['1st_flr_sf']] +\
                   X[:, idx['2nd_flr_sf']]
        
        qual_sf_total = X[:, idx["overall_qual"]] * total_sf
        #kitchen_total = X[:, idx["kitech_qual"]]
        
        return np.c_[X, qual_squared, qual_m2, garage_int, total_sf,
                    qual_sf_total]

### Ahora se entenderá más claro qué es esto. 

Básicamente, esta clase nos ayuda a definir las nuevas variables que queremos agregar y es algo que usaremos en nuestro pipeline. El siguiente paso es crear un pipeline para las variables numéricas. 

In [333]:
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('attribs_adder', CombinedAttributesAdder2()),
    ('std_scaler', StandardScaler()),
])

## ¿Cómo se usa esto? 

Seleccionemos primero las variables numéricas: 

In [334]:
num_attribs = X.select_dtypes(include = "number").columns.values
num_attribs

array(['Lot Area', 'Overall Qual', 'Overall Cond', 'Year Built',
       'Year Remod/Add', 'BsmtFin SF 1', 'BsmtFin SF 2', 'Bsmt Unf SF',
       'Total Bsmt SF', '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF',
       'Gr Liv Area', 'Bsmt Full Bath', 'Bsmt Half Bath', 'Full Bath',
       'Half Bath', 'Bedroom AbvGr', 'Kitchen AbvGr', 'TotRms AbvGrd',
       'Fireplaces', 'Garage Cars', 'Garage Area', 'Wood Deck SF',
       'Open Porch SF', 'Enclosed Porch', '3Ssn Porch', 'Screen Porch',
       'Pool Area', 'Misc Val', 'Mo Sold', 'Yr Sold'], dtype=object)

In [335]:
len(num_attribs)

32

In [336]:
data_numeric = X[num_attribs]
numeric_transformed = numeric_pipeline.fit_transform(data_numeric)

In [337]:
numeric_transformed.shape

(1208, 35)

Son las 32 variables numéricas más otras 3 que se agregaron en la clase de AttributesAdder2. 

### Pipeline completo (numérico más categórico).

Una vez que tenemos esto, en realidad completar el pipeline no es tan difícil. 

In [338]:
cat_attribs = X.select_dtypes(include = 'object').columns.values
len(cat_attribs)

20

In [339]:
full_pipeline = ColumnTransformer([
    ("numeric", numeric_pipeline, num_attribs),
    ("categorical", OneHotEncoder(handle_unknown='ignore'), cat_attribs),
])

In [340]:
data_prepared = full_pipeline.fit_transform(X)

In [341]:
data_prepared.shape #es lo que usaremos para predecir

(1208, 148)

Terminamos teniendo 196 variables porque son demasiadas categóricas transformamdas en OneHotEncoding() (recordemos que por cada n clases, se crean n variables adicionales). 

Después podemos quitar más variables. Lo importante ahora es estandarizar todo. 

## Entrenando el modelo con validación cruzada 

(Primero quiero poner esta parte del pipeline y después hacer lo de grid searching, o la búsqueda de los mejores hiperparámetros). 

In [342]:
ridge_reg = Ridge()

In [343]:
scores = cross_val_score(ridge_reg, data_prepared, Y, scoring = "neg_mean_squared_error", cv = 5)
ridge_reg_scores = np.sqrt(-scores)

In [344]:
ridge_reg_scores

array([21530.57569061, 22571.76338959, 22714.28575127, 27371.64881815,
       23330.6046707 ])

In [314]:
#import joblib
#joblib.dump(modelo_uno, "modelo_uno.pkl")
#y luego lo cargas: 
#modelo_cargado = joblib.load("modelo_uno.pkl")

# Grid Search. 

In [345]:
param_grid = [
    {'alpha': [.001, .01, .1, 1, 10, 15, 20], 
     'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']}
]

In [346]:
ridge_reg = Ridge()

In [347]:
grid_search = GridSearchCV(ridge_reg, param_grid, cv = 5, scoring = "neg_mean_squared_error",
                          return_train_score = True)

In [348]:
grid_search.fit(data_prepared, Y)

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'alpha': [0.001, 0.01, 0.1, 1, 10, 15, 20], 'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

In [349]:
grid_search.best_params_

{'alpha': 10, 'solver': 'saga'}

In [350]:
def clean_test_dataset(path):
    data = pd.read_csv(path)
    #Arreglando tipos de datos
    data['MS SubClass'] = data['MS SubClass'].astype('category')
    data = data.drop(columns = ['MS Zoning', 'Lot Frontage', 'Alley', 'Mas Vnr Type', 'Mas Vnr Area',
     'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin Type 2', 'Electrical',
     'Fireplace Qu', 'Garage Type', 'Garage Yr Blt', 'Garage Finish', 'Garage Qual', 'Garage Cond',
     'Pool QC', 'Fence', 'Misc Feature', 'Condition 1', 'Condition 2', 'Exterior 1st', 'Exterior 2nd',
    'Heating', 'Heating QC'])
    return data

In [351]:
datos_prueba = clean_test_dataset('casas_prueba.csv')

In [352]:
prueba_prepared = full_pipeline.transform(datos_prueba.iloc[:,:-1])

In [353]:
final_model = grid_search.best_estimator_
final_predictions = final_model.predict(prueba_prepared)

In [354]:
len(final_predictions)

1203

In [355]:
submissions = pd.DataFrame({'id': [e for e in range(1,1204)], 'SalePrice': final_predictions})

In [356]:
submissions.head()

,id,SalePrice
0,1,239311.140338
1,2,218319.318540
2,3,142500.003500
3,4,287138.633240
4,5,185401.371870


In [358]:
submissions.to_csv('submissions_21_octubre.csv')

### Placeholder. 

Vamos a seguir explorando las etiquetas que no estén. Quizá me tome unos 30 minutos pero está bien. después de eso ya podemos empezar a predecir con nuevos modelos :) Y le puedo avanzar en tiempos libres. Me quedé a punto de tirar "Heating" tanto hasta arriba como abajo. 

In [293]:
datos_prueba.columns

Index(['MS SubClass', 'Lot Area', 'Street', 'Lot Shape', 'Land Contour',
       'Utilities', 'Lot Config', 'Land Slope', 'Neighborhood', 'Bldg Type',
       'House Style', 'Overall Qual', 'Overall Cond', 'Year Built',
       'Year Remod/Add', 'Roof Style', 'Roof Matl', 'Exter Qual', 'Exter Cond',
       'Foundation', 'BsmtFin SF 1', 'BsmtFin SF 2', 'Bsmt Unf SF',
       'Total Bsmt SF', 'Heating', 'Heating QC', 'Central Air', '1st Flr SF',
       '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Bsmt Full Bath',
       'Bsmt Half Bath', 'Full Bath', 'Half Bath', 'Bedroom AbvGr',
       'Kitchen AbvGr', 'Kitchen Qual', 'TotRms AbvGrd', 'Functional',
       'Fireplaces', 'Garage Cars', 'Garage Area', 'Paved Drive',
       'Wood Deck SF', 'Open Porch SF', 'Enclosed Porch', '3Ssn Porch',
       'Screen Porch', 'Pool Area', 'Misc Val', 'Mo Sold', 'Yr Sold',
       'Sale Type', 'Sale Condition', 'id'],
      dtype='object')

In [324]:
datos_prueba['Heating QC'].value_counts()

Ex    592
TA    367
Gd    202
Fa     41
Po      1
Name: Heating QC, dtype: int64

# Resumen (hasta el momento).  

In [ ]:
#primero usamos una función 